In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
sys.path.append("../..")
sys.path.append("..")
sys.path.append(os.getcwd())
import re
import datetime

import math
import scipy

from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import seaborn as sns
sns.set()
sns.set(palette="bright")
sns.set_style("whitegrid")

from sklearn import linear_model
from cycler import cycler
from celluloid import Camera
from IPython.display import HTML

import importlib
import warnings
import numpy as np
import pandas as pd
import copy
import random
from tslib.src import tsUtils
from tslib.tests import testdata
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation, DBSCAN #For clustering
from sklearn.mixture import GaussianMixture #For GMM clustering
from filter_data import *
from load_and_clean import *
import hdbscan
%matplotlib inline

In [3]:
# sys.path.append('../../prgds/src')
# from apf.models.prgds import PRGDS

In [4]:
import warnings
warnings.filterwarnings('ignore')

## Pre-processing data

In [5]:
#global settings for graphs        
cc = (cycler(linestyle=['-', '--', '-.',':']) * cycler(color=list('brgcmyk')))
plt.rc('lines', linewidth=2)
plt.rc('axes', prop_cycle=cc)
plt.rcParams['figure.figsize'] = (8.0, 6.0)

plt.rcParams["animation.html"] = "html5"

size=14
params = {'legend.fontsize': 'large',
          'figure.figsize': (20,8),
          'axes.labelsize': size,
          'axes.titlesize': size,
          'xtick.labelsize': size*0.75,
          'ytick.labelsize': size*0.75,
          'axes.titlepad': 25}
plt.rcParams.update(params)
#plt.rcParams.update({'font.size': 14})

In [6]:
#Load and clean all raw data
update_data()
us = load_clean('NYTimes US')
uscases, usdeaths, usstates = load_clean('NYTimes states')
uscounties_cases, uscounties_deaths, uscounties = load_clean('NYTimes counties')
global_cases, global_deaths = load_clean('JHU global')
us_covid_jhu_cases, us_covid_jhu_deaths = load_clean('JHU US')
# mobility_data_apple, mobility_data_google, google_social = load_clean('mobility')
sd_data = load_clean('IHME intervention')
all_population, country_population, usstates_population, uscounties_population = load_clean('population')

uscases_pop_adjusted, usdeaths_pop_adjusted = create_population_adjusted_data(uscases, all_population), \
                                    create_population_adjusted_data(usdeaths, all_population, show_exception = True)
global_cases_pop_adjusted, global_deaths_pop_adjusted = create_population_adjusted_data(global_cases, all_population), \
                                    create_population_adjusted_data(global_deaths, all_population, show_exception = True)

uscounties_cases_pop_adjusted = create_population_adjusted_data(uscounties_cases, all_population, show_exception = True, county = True)
uscounties_deaths_pop_adjusted = create_population_adjusted_data(uscounties_deaths, all_population, county = True)

These countries/region do not have population data ['Virgin Islands']
These countries/region do not have population data ['Bahamas', 'Burma', 'Cabo Verde', 'Congo (Brazzaville)', 'Congo (Kinshasa)', "Cote d'Ivoire", 'Diamond Princess', 'Eswatini', 'Gambia', 'Holy See', 'Kyrgyzstan', 'MS Zaandam', 'North Macedonia', 'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Vincent and the Grenadines', 'Slovakia', 'Syria', 'Taiwan*', 'West Bank and Gaza', 'Western Sahara']
These countries/region do not have population data ['Adjuntas-Puerto Rico', 'Aguada-Puerto Rico', 'Aguadilla-Puerto Rico', 'Aguas Buenas-Puerto Rico', 'Aibonito-Puerto Rico', 'Anasco-Puerto Rico', 'Anchorage-Alaska', 'Arecibo-Puerto Rico', 'Arroyo-Puerto Rico', 'Barceloneta-Puerto Rico', 'Barranquitas-Puerto Rico', 'Bayamon-Puerto Rico', 'Cabo Rojo-Puerto Rico', 'Caguas-Puerto Rico', 'Camuy-Puerto Rico', 'Canovanas-Puerto Rico', 'Carolina-Puerto Rico', 'Catano-Puerto Rico', 'Cayey-Puerto Rico', 'Ceiba-Puerto Rico', 'Ciales-Puerto

In [8]:
#Filter data 
# This part might be to split and explain
# This part might be to split and explain
t = 7

global_deaths_filtered = create_filtered_data(global_deaths, 500 )
#the last column of social distancing dataset is the last social distancing measure used by a location
educational_distancing = get_social_distancing(sd_data, sd_data.columns[-1])

all_data = pd.concat([global_deaths, usdeaths, uscounties_deaths], axis=1)  
all_data_daily = create_rolling_data(all_data, t)

all_data_cases = pd.concat([global_cases, uscases, uscounties_cases], axis=1) 
all_data_cases_daily = create_rolling_data(all_data_cases, t)

all_cases_data_pop = pd.concat([uscases_pop_adjusted,global_cases_pop_adjusted, uscounties_cases_pop_adjusted], axis = 1)
all_cases_data_pop_daily = create_rolling_data(all_cases_data_pop, t)

all_death_data_pop = pd.concat([usdeaths_pop_adjusted,global_deaths_pop_adjusted, uscounties_deaths_pop_adjusted], axis = 1)
all_death_data_pop_daily = create_rolling_data(all_death_data_pop, t)

_, daily_global, intervention_date_global = create_intervention_adjusted_data(global_deaths, educational_distancing, t)
_, daily_us, intervention_date_us = create_intervention_adjusted_data(usdeaths, educational_distancing, t)
_, daily_uscounties, intervention_date_uscounties = create_intervention_adjusted_data(uscounties_deaths, educational_distancing, t)

cases_intervention_adjusted, cases_intervention_adjusted_daily, intervention_date_case = create_intervention_adjusted_data(all_data_cases, educational_distancing, t)
deaths_intervention_adjusted, deaths_intervention_adjusted_daily, intervention_date_death = create_intervention_adjusted_data(all_data, educational_distancing, t)

all_cases_data_pop_adjusted, all_cases_data_pop_adjusted_daily, _ = create_intervention_adjusted_data(all_cases_data_pop, educational_distancing, t)
all_death_data_pop_adjusted, all_death_data_pop_adjusted_daily, _ = create_intervention_adjusted_data(all_death_data_pop, educational_distancing, t)

last date
Exceptions are []


In [9]:
masks_mandated = ['New York', 'Maine', 'Maryland', 'Virginia', 'New Mexico', 'California', 'Michigan', 'Illinois', 'Massachusetts','Delaware', 'Rhode Island']
masks_recommended = ['Montana', 'Idaho', 'Utah', 'Arizona', 'North Dakota', 'South Dakota', 'Kansas', 'Oklahoma', 
                     'Texas', 'North Carolina', 'South Carolina', 'West Virginia', 'Wisconsin','Iowa', 'Missouri', 'Alaska']
eu_countries = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Denmark',
   'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland',
   'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands',
   'Poland', 'Portugal', 'Romania', 'Slovenia', 'Spain', 'Sweden']

northeast = ["Connecticut","Maine","Massachusetts","New Hampshire",
             "Rhode Island","Vermont","New Jersey","New York",
             "Pennsylvania", "Delaware","District of Columbia","Maryland"]
midwest = ["Indiana","Illinois","Michigan","Ohio","Wisconsin",
             "Iowa","Kansas","Minnesota","Missouri","Nebraska",
             "North Dakota","South Dakota"]
south = ["Florida","Georgia",
            "North Carolina","South Carolina","Virginia",
            "West Virginia","Alabama","Kentucky","Mississippi",
            "Tennessee","Arkansas","Louisiana","Oklahoma","Texas"]

west= ["Arizona","Colorado","Idaho","New Mexico","Montana",
            "Utah","Nevada","Wyoming","Alaska","California",
            "Hawaii","Oregon","Washington"]

california_rollback = ["Fresno-California", "Imperial-California", "Kern-California", "Kings-California", "Los Angeles-California", "San Joaquin-California" , "Tulare-California",
                      "Riverside-California", "Sacramento-California", "San Bernardino-California", "Santa Barbara-California", "Santa Clara-California", "Contra Costa-California", "Stanislaus-California", 
                       "Ventura-California"]

google_counties = ['San Mateo-California', 'San Francisco-California', 'Santa Clara-California']

### Population adjusted counterfactual analysis

In [33]:
placebo_study = False
if not placebo_study:
    target_states = ['Florida']
    donor_states = northeast
else:
    target_states = ['Massachusetts']
    donor_states = northeast

df = all_cases_data_pop_daily[uscounties_cases_pop_adjusted.columns].loc['2020-03-08':]

donor_counties = []
for state in donor_states:
    donor_counties += list(uscounties.loc[uscounties.state == state].county_state.unique())

target_counties = []
for state in target_states:
    target_counties += list(uscounties.loc[uscounties.state == state].county_state.unique())

df1 = df[df.columns[(df > 10).any()]].copy()
donor_counties = list(df1.columns[df1.columns.isin(donor_counties)])
target_counties = list(df1.columns[df1.columns.isin(target_counties)])

In [34]:
end_t = df.shape[0]
start_t = end_t - 72

trainDF = df.iloc[:start_t,:]
testDF = df.iloc[start_t+1:end_t,:]

targets = target_counties.copy()
donors = donor_counties.copy()

num_rows_to_plot = int(len(targets)/4) + 1
plt.rcParams['figure.figsize'] = (24, num_rows_to_plot*5.0)
plt.rcParams.update({'font.size': 10})

# print("Counterfactual analysis of Mask recommended counties based on a donor pool of Mask mandated counties")
# print("Intervention date: ", testDF.index[0])
# for i, county in enumerate(targets):
#     other_counties = df.columns[(df.iloc[start_t] < df[county].iloc[start_t] + 0.5 * df[county].iloc[start_t]) & 
#                                 (df.iloc[start_t] > df[county].iloc[start_t] - 0.5 * df[county].iloc[start_t])]
#     other_counties = list(other_counties[other_counties.isin(donors)])
#     other_counties = list(filter(lambda x: x != county, other_counties))
#     all_rows = list.copy(other_counties) + [county]
    
#     rscModel = RobustSyntheticControl(seriesToPredictKey=county, 
#                                       kSingularValuesToKeep=4, 
#                                       M=len(trainDF), 
#                                       probObservation=1.0, 
#                                       modelType='svd', 
#                                       svdMethod='numpy', 
#                                       otherSeriesKeysArray=other_counties)
#     rscModel.fit(trainDF)

#     denoisedDF = rscModel.model.denoisedDF()
#     predictions = np.dot(testDF[other_counties].values, rscModel.model.weights)
    
#     model_fit = np.dot(trainDF[other_counties][:], rscModel.model.weights)
#     fit_error = mse(model_fit, df[county][:start_t])
#     prediction_error = mse(predictions, df[county][start_t+1:])

#     ax = plt.subplot(num_rows_to_plot, 4, i+1)
#     ax.xaxis.set_major_locator(ticker.MultipleLocator(50))
#     ax.set_title(county + ' Donors: ' + str(len(other_counties)), fontsize=12, pad=10)
#     plt.plot(np.concatenate((model_fit, predictions)), label='Model+Counterfactual')
#     plt.plot(df[county], label='Actual')
#     plt.ylabel('Moving Average Daily Cases Per Million')
#     plt.axvline(x=df.index[start_t], color='k', linestyle='--', linewidth=2)
#     plt.legend(loc='upper right')
# plt.show()


In [35]:
trainDF

,Abbeville-South Carolina,Acadia-Louisiana,Accomack-Virginia,Ada-Idaho,Adair-Iowa,Adair-Kentucky,Adair-Missouri,Adair-Oklahoma,Adams-Colorado,Adams-Idaho,...,York-South Carolina,York-Virginia,Young-Texas,Yuba-California,Yukon-Koyukuk Census Area-Alaska,Yuma-Arizona,Yuma-Colorado,Zapata-Texas,Zavala-Texas,Ziebach-South Dakota
date,,,,,,,,,,,,,,,,,,,,,
2020-03-08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-03-09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-03-10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-03-11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-03-12,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.552189,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-03-13,0.000000,0.000000,0.000000,0.296638,0.000000,0.000000,0.000000,0.000000,0.828284,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-03-14,0.000000,0.000000,0.000000,0.593277,0.000000,0.000000,0.000000,0.000000,1.656568,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-03-15,0.000000,0.000000,0.000000,0.593277,0.000000,0.000000,0.000000,0.000000,1.656568,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-03-16,0.000000,0.000000,0.000000,0.593277,0.000000,0.000000,0.000000,0.000000,2.208757,0.000000,...,0.000000,2.092225,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [37]:
from sklearn.decomposition import PCA

train_split = 70

train_data_TV = np.array(trainDF[target_counties + donor_counties]).astype(int)
train_data_TD = np.array(trainDF[donor_counties]).astype(int)[:train_split]
train_data_TP = np.array(trainDF[target_counties]).astype(int)[:train_split]

# test_data_TD = np.array(testDF[donor_counties]).astype(int)
# test_data_TP = np.array(testDF[target_counties]).astype(int)

test_data_TD = np.array(trainDF[donor_counties]).astype(int)[train_split:]
test_data_TP = np.array(trainDF[target_counties]).astype(int)[train_split:]



pca = PCA(n_components=20)
pca.fit(train_data_TV)
pca_Phi_VK = pca.components_.T
pca_Phi_PK = pca_Phi_VK[:len(target_counties)]
pca_Phi_DK = pca_Phi_VK[len(target_counties):]

up_to = None

# reg = linear_model.Ridge(alpha=0)
# reg.fit(train_data_TD.T, test_data_TD[:up_to].T)
# print(reg.score(train_data_TD.T, test_data_TD[:up_to].T))
# print(reg.score(train_data_TP.T, test_data_TP[:up_to].T))

reg = linear_model.Ridge(alpha=0)
reg.fit(pca_Phi_DK, test_data_TD[:up_to].T)
print(reg.score(pca_Phi_DK, test_data_TD[:up_to].T))

print(reg.score(pca_Phi_PK, test_data_TP[:up_to].T))

pca_reconstruct = (pca.mean_  + pca.transform(train_data_TV).dot(pca_Phi_VK.T)).T  # all units x train timesteps
pca_predictions = reg.predict(pca_Phi_PK)                                          # targets x test timesteps

pca_model_fit = pca_reconstruct[:len(target_counties)]

0.993978470044433
0.9595004146531887


In [34]:
# from sklearn.decomposition import PCA

# train_data_TV = np.array(trainDF[target_counties + donor_counties]).astype(int)
# train_data_TD = np.array(trainDF[donor_counties]).astype(int)
# train_data_TP = np.array(trainDF[target_counties]).astype(int)

# test_data_TD = np.array(testDF[donor_counties]).astype(int)
# test_data_TP = np.array(testDF[target_counties]).astype(int)

# pca = PCA(n_components=20)
# pca.fit(train_data_TV)
# pca_Phi_VK = pca.components_.T
# pca_Phi_PK = pca_Phi_VK[:len(target_counties)]
# pca_Phi_DK = pca_Phi_VK[len(target_counties):]

# up_to = None

# # reg = linear_model.Ridge(alpha=0)
# # reg.fit(train_data_TD.T, test_data_TD[:up_to].T)
# # print(reg.score(train_data_TD.T, test_data_TD[:up_to].T))
# # print(reg.score(train_data_TP.T, test_data_TP[:up_to].T))

# reg = linear_model.Ridge(alpha=0)
# reg.fit(pca_Phi_DK, test_data_TD[:up_to].T)
# print(reg.score(pca_Phi_DK, test_data_TD[:up_to].T))

# print(reg.score(pca_Phi_PK, test_data_TP[:up_to].T))

# pca_reconstruct = (pca.mean_  + pca.transform(train_data_TV).dot(pca_Phi_VK.T)).T  # all units x train timesteps
# pca_predictions = reg.predict(pca_Phi_PK)                                          # targets x test timesteps

# pca_model_fit = pca_reconstruct[:len(target_counties)]

In [38]:
df = trainDF
end_t = df.shape[0]
start_t = end_t - train_split

In [25]:
print("Counterfactual analysis of Mask recommended counties based on a donor pool of Mask mandated counties")
print("Intervention date: ", testDF.index[0])
for i, county in enumerate(targets):
    other_counties = df.columns[(df.iloc[start_t] < df[county].iloc[start_t] + 0.5 * df[county].iloc[start_t]) & 
                                (df.iloc[start_t] > df[county].iloc[start_t] - 0.5 * df[county].iloc[start_t])]
    other_counties = list(other_counties[other_counties.isin(donors)])
    other_counties = list(filter(lambda x: x != county, other_counties))
    all_rows = list.copy(other_counties) + [county]
    

    predictions = pca_predictions[i]
    model_fit = pca_model_fit[i]
    
    
    fit_error = mse(model_fit, df[county][:start_t])
    prediction_error = mse(predictions, df[county][start_t+1:])

    ax = plt.subplot(num_rows_to_plot, 4, i+1)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(50))
    ax.set_title(county + ' Donors: ' + str(len(other_counties)), fontsize=12, pad=10)
    plt.plot(np.concatenate((model_fit, predictions)), label='Model+Counterfactual', lw=3, color='Blue')
    plt.plot(df[county], label='Actual', lw=3, color='Red')
    plt.plot(np.concatenate((model_fit, predictions)), lw=3, color='Blue')
    plt.ylabel('Moving Average Daily Cases Per Million')
    plt.axvline(x=df.index[start_t], color='k', linestyle='--', linewidth=2)
    plt.legend(loc='upper right')
plt.show()

Counterfactual analysis of Mask recommended counties based on a donor pool of Mask mandated counties
Intervention date:  2020-06-11


ValueError: operands could not be broadcast together with shapes (94,) (24,) 

In [ ]:
# n_components = 20
# data_TV = np.array(trainDF[target_counties + donor_counties]).astype(int)
# assert (data_TV >= 0).all()

# model = PRGDS(data_shp=data_TV.shape,
#               core_shp=(n_components,),
#               time_mode=0,
#               stationary=1,
#               eps_nu=1.0,
#               mtx_is_dirichlet=[],
#               seed=617,
#               n_threads=1)

# n_samples = 20
# thin_rate = 50
# n_burn_in = 1000

# model.fit(data_TV,
#           schedule=model.get_default_schedule(),
#           verbose=100,
#           n_itns=n_burn_in,
#           initialize=True)

# for s in range(n_samples):
#     model.fit(data_TV,
#               schedule=model.get_default_schedule(),
#               verbose=50,
#               n_itns=thin_rate,
#               initialize=False)
#     state = dict(model.get_state())
#     np.savez_compressed('samples/state_%d.npz' % s, **state)
    

In [ ]:
forecast_TV = model.forecast(n_timesteps=59)
forecast_TP = forecast_TV[:,:len(target_counties)]
forecast_TD = forecast_TV[:,len(target_counties):]

np.sqrt(np.mean((forecast_TP[:1]-test_data_TP[:1])**2))
# np.sqrt(np.mean((forecast_TD-test_data_TD)**2))

# np.sqrt(np.mean((model.reconstruct()-data_TV)**2))

In [ ]:
from path import Path

for posterior_sample_path in Path('samples').files('state*.npz'):
    posterior_sample = np.load(posterior_sample_path)
    
    Theta_TK = posterior_sample['Theta_TK']    # TxK (time x latent components) matrix
    Phi_VK = posterior_sample['mtx_MKD'][1].T  # VxK (unit x latent components) matrix
    
    Phi_PK = Phi_VK[:len(target_counties)]      # (target units x latent components) sub-matrix
    Phi_DK = Phi_VK[len(target_counties):]      # (donor units x latent components) sub-matrix
    
    pass

In [ ]:
from sklearn import linear_model

train_data_TD = np.array(trainDF[donor_counties]).astype(int)
train_data_TP = np.array(trainDF[target_counties]).astype(int)

test_data_TD = np.array(testDF[donor_counties]).astype(int)
test_data_TP = np.array(testDF[target_counties]).astype(int)

up_to = None

reg = linear_model.Ridge(alpha=0)
reg.fit(train_data_TD.T, test_data_TD[:up_to].T)
print(reg.score(train_data_TD.T, test_data_TD[:up_to].T))
print(reg.score(train_data_TP.T, test_data_TP[:up_to].T))

reg = linear_model.Ridge(alpha=0)
reg.fit(Phi_DK, test_data_TD[:up_to].T)
print(reg.score(Phi_DK, test_data_TD[:up_to].T))

print(reg.score(Phi_PK, test_data_TP[:up_to].T))


prgds_reconstruct = model.reconstruct().T  # all units x train timesteps
prgds_predictions = reg.predict(Phi_PK)    # targets x test timesteps

prgds_model_fit = prgds_reconstruct[:len(target_counties)]

In [ ]:
prgds_model_fit.shape, prgds_predictions.shape

In [ ]:
Theta_TK.dot(Phi_VK.T)

In [ ]:
model.reconstruct()

In [ ]:
def get_predictions_and_fit(posterior_sample, test_data_TD, n_targets):
    mtx_KT = posterior_sample['mtx_MKD'][0][:,:data_TV.shape[0]]
    mtx_KV = posterior_sample['mtx_MKD'][1][:,:data_TV.shape[1]]
    recon_VT = mtx_KT.T.dot(mtx_KV).T
    model_fit = recon_VT[:n_targets]
    
    Phi_PK = mtx_KV.T[:len(targets)]
    Phi_DK = mtx_KV.T[len(targets):]
    
    reg = linear_model.Ridge(alpha=0)
    reg.fit(Phi_DK, test_data_TD.T)
    predictions = reg.predict(Phi_PK)
    
    return predictions, model_fit

n_samples = len(Path('samples').files('state*.npz'))
first_sample = np.load(Path('samples').files('state*.npz')[0])
first_predictions, first_model_fit = get_predictions_and_fit(first_sample, test_data_TD, len(targets))

predictions_SPT = np.zeros((n_samples,) + first_predictions.shape)
model_fit_SPT = np.zeros((n_samples,) + first_model_fit.shape)

for s, posterior_sample_path in enumerate(Path('samples').files('state*.npz')):
    posterior_sample = np.load(posterior_sample_path)
    predictions_PT, model_fit_PT = get_predictions_and_fit(posterior_sample, test_data_TD, len(targets))
    predictions_SPT[s] = predictions_PT
    model_fit_SPT[s] = model_fit_PT

avg_predictions_PT = predictions_SPT.mean(axis=0)
avg_model_fit_PT = model_fit_SPT.mean(axis=0)

print("Counterfactual analysis of Mask recommended counties based on a donor pool of Mask mandated counties")
print("Intervention date: ", testDF.index[0])
for i, county in enumerate(targets):
    print(county)
    other_counties = df.columns[(df.iloc[start_t] < df[county].iloc[start_t] + 0.5 * df[county].iloc[start_t]) & 
                                (df.iloc[start_t] > df[county].iloc[start_t] - 0.5 * df[county].iloc[start_t])]
    other_counties = list(other_counties[other_counties.isin(donors)])
    other_counties = list(filter(lambda x: x != county, other_counties))
    all_rows = list.copy(other_counties) + [county]
    
    ax = plt.subplot(num_rows_to_plot, 4, i+1)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(50))
    ax.set_title(county + ' Donors: ' + str(len(other_counties)), fontsize=12, pad=10)




    predictions = avg_predictions_PT[i]
    model_fit = avg_model_fit_PT[i]
    plt.plot(np.concatenate((model_fit, predictions)), label='Model+Counterfactual', color='Blue', lw=3)
    plt.plot(df[county], label='Actual',color='Red', lw=1)
    for s in range(n_samples):
        predictions = predictions_SPT[s,i]
        model_fit = model_fit_SPT[s,i]
        plt.plot(np.concatenate((model_fit, predictions)), alpha=0.1, color='Blue', lw=3)
    plt.plot(np.concatenate((model_fit, predictions)), color='Blue', lw=3)
    plt.plot(df[county], color='Red', lw=3)
    

    plt.ylabel('Moving Average Daily Cases Per Million')
    
    plt.axvline(x=df.index[start_t], color='k', linestyle='--', linewidth=2)
    
    plt.legend(loc='upper right')
plt.show()